In [1]:
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold 
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV

device = 'cpu'

#Code loosly based on https://datascience.stackexchange.com/questions/56804/sckit-learn-cross-validation-and-model-retrain
#And our exercise 1a

In [2]:
train = pd.read_csv(r'C:\Users\erics\Documents\Programme\IntroML\Task1\task1b_ql4jfi6af0\train.csv')

X = train.iloc[:,2:]
Y = train.iloc[:,1]
ID = train.iloc[:,0]
pd.set_option("display.precision", 7)
print(X)
print(Y)
print(X.shape)
print(Y.shape)

       x1    x2    x3    x4    x5
0    0.02  0.05 -0.09 -0.43 -0.08
1   -0.13  0.11 -0.08 -0.29 -0.03
2    0.08  0.06 -0.07 -0.41 -0.03
3    0.02 -0.12  0.01 -0.43 -0.02
4   -0.14 -0.12 -0.08 -0.02 -0.08
..    ...   ...   ...   ...   ...
695  0.31 -0.09  0.04 -0.09  0.03
696 -0.26 -0.01  0.02 -0.40  0.05
697 -0.27 -0.22 -0.01 -0.32 -0.05
698  0.19  0.11 -0.05 -0.27 -0.04
699 -0.09 -0.09 -0.06 -0.41 -0.06

[700 rows x 5 columns]
0     -6.8226793
1     -6.3262905
2     -9.3027282
3     -7.3718925
4     -6.0276470
         ...    
695   -6.1473937
696   -6.2148989
697   -6.8413411
698   -6.5233707
699   -5.6766827
Name: y, Length: 700, dtype: float64
(700, 5)
(700,)


Wir verlieren etwas präzision durch das Laden in den DataFrame (durch float precision)

In [3]:
def transform(X):
    l1 = X
    l2 = X*X
    l3 = np.exp(X)
    l4 = np.cos(X)
    const = np.ones((X.shape[0],1))
    return np.concatenate((l1, l2, l3, l4, const), axis=1)

In [4]:
transformer = FunctionTransformer(transform, validate=True)

In [5]:
#X_Test = np.array([[1, 2, 3, 4, 5], [1, 1,1, 1,1], [1, 1,1, 1,1], [1, 1,1, 1,1]])
#print(transformer.transform(X_Test))

In [6]:
X_trans = transformer.transform(X)
print(X_trans.shape)

(700, 21)


In [13]:
param_grid = {'alpha' : np.logspace(-2, 20, num=500)}
kf = KFold(n_splits=10)
model = Ridge(alpha=1)

grid_search = GridSearchCV(model, param_grid, cv=kf, scoring='neg_root_mean_squared_error', return_train_score=True)

In [14]:
best_model = grid_search.fit(X_trans, Y)
print("Lowest RMSE:" + str(best_model.best_score_))
best_alpha = best_model.best_estimator_
print("With alpha: " + str(best_alpha))

coefficiants = best_model.best_estimator_.coef_
print("And coefficiants. ")
print(coefficiants)

Lowest MSE: -1.9544610282420538
With alpha: Ridge(alpha=5.413360302965381)
And coefficiants. 
[-0.07002198 -0.67209581 -0.81806507 -0.05550941 -0.03943274 -0.23734516
  0.18733674  0.16142086 -0.0323701   0.1049385  -0.19701087 -0.58247842
 -0.74521486 -0.07995748  0.00320602  0.11794449 -0.0931548  -0.08017685
  0.01528865 -0.05082425  0.        ]


store


In [12]:
dictionary = {'Coefficiants': coefficiants}
avg_acc_df = pd.DataFrame(dictionary )
print(avg_acc_df)
avg_acc_df.to_csv(r'C:\Users\erics\Documents\Programme\IntroML\Task1\submission1b.csv', index=False, header=False)

    Coefficiants
0     -0.0782118
1     -0.6492389
2     -0.7827070
3     -0.0574910
4     -0.0349332
5     -0.2156051
6      0.1728397
7      0.1526187
8     -0.0267091
9      0.0945264
10    -0.1936406
11    -0.5668747
12    -0.7137819
13    -0.0784624
14     0.0034448
15     0.1071381
16    -0.0859434
17    -0.0758117
18     0.0125573
19    -0.0457747
20     0.0000000
